# Cleaning the dataset
- This notebook was used to choose only one genre for every song in the dataset that appeared with multiple genres
- Genres were chosen based on the manhattan distance of the numerical vector of the song w.r.t the mean vectors of each genre
    - Genre with the smallest distance to the song was chosen
- Generated a .csv file from the resulting 'clean' dataframe and stored it under `data/spotify_clean.csv`

In [1]:
import pandas as pd

songs = pd.read_csv('data/spotify_data.csv', index_col=[0])
# songs.drop_duplicates(inplace=True, keep='first')
songs.dropna(inplace=True)
explicit = songs['explicit'].astype(int).to_numpy()
songs['explicit'] = explicit
unique_trackids = songs.drop_duplicates(subset='track_id', keep='first')
unique_trackids = unique_trackids.set_index(['track_id'])
same_trackid = songs[songs.duplicated(subset=['track_id'], keep=False)].sort_values(by='track_id')
same_trackid.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
15028,001APMDOl3qtx1526T11n1,Pink Sweat$;Kirby,New RnB,Better,0,176320,0,0.613,0.471,1,-6.644,0,0.1070,0.31600,0.000001,0.1170,0.406,143.064,4,chill
103211,001APMDOl3qtx1526T11n1,Pink Sweat$;Kirby,New RnB,Better,0,176320,0,0.613,0.471,1,-6.644,0,0.1070,0.31600,0.000001,0.1170,0.406,143.064,4,soul
85578,001YQlnDSduXd5LgBd66gT,Soda Stereo,Soda Stereo (Remastered),El Tiempo Es Dinero - Remasterizado 2007,38,177266,0,0.554,0.921,2,-4.589,1,0.0758,0.01940,0.088100,0.3290,0.700,183.571,1,punk-rock
100420,001YQlnDSduXd5LgBd66gT,Soda Stereo,Soda Stereo (Remastered),El Tiempo Es Dinero - Remasterizado 2007,38,177266,0,0.554,0.921,2,-4.589,1,0.0758,0.01940,0.088100,0.3290,0.700,183.571,1,ska
91801,003vvx7Niy0yvhvHt4a68B,The Killers,Hot Fuss,Mr. Brightside,86,222973,0,0.352,0.911,1,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,4,rock


In [30]:
by_track_id = same_trackid.groupby(['track_id'], as_index=False).agg({'track_genre': ' '.join})
print(by_track_id)

                     track_id                    track_genre
0      001APMDOl3qtx1526T11n1                     chill soul
1      001YQlnDSduXd5LgBd66gT                  punk-rock ska
2      003vvx7Niy0yvhvHt4a68B      rock alternative alt-rock
3      004h8smbIoAkUNDJvVKwkG                        emo sad
4      006rHBBNLJMpQs8fRC2GDe         sertanejo pagode forro
...                       ...                            ...
16636  7ztSVy67w9rXpKg5L2zN5l                   indie indian
16637  7zubR9uYAWjb5KPZTMm85e  latino latin reggaeton reggae
16638  7zumacGldlmxpoP8bpaeLe            power-pop synth-pop
16639  7zv2vmZq8OjS54BxFzI2wM          metalcore death-metal
16640  7zwn1eykZtZ5LODrf7c0tS      alt-rock alternative rock

[16641 rows x 2 columns]


In [31]:
from sklearn.preprocessing import StandardScaler
string_columns = ['track_name', 'track_id', 'album_name', 'artists']
numerical_data = songs.drop(columns=string_columns)
by_genre = numerical_data.groupby('track_genre').mean()
scaler = StandardScaler()
scaler.fit(numerical_data.values)
by_genre[by_genre.columns] = scaler.transform(by_genre.values)

In [32]:
import numpy as np

id_to_selected_genre = {}
for track_id, genres in by_track_id.to_numpy():
    record_values = unique_trackids.loc[track_id]
    numerical_values = record_values.drop(index=['artists', 'album_name', 'track_name', 'track_genre'])
    scaled = scaler.transform(numerical_values.to_numpy().reshape(1, -1))
    song_vector = scaled.flatten()
    distances = []
    # Compute distance from each genre to the current song
    for genre in genres.split(" "):
        genre_vector = by_genre.loc[genre]
        distance = np.linalg.norm(song_vector - genre_vector, ord=1)
        distances.append((genre, distance))
    # Select genre with the smallest distance
    selected_genre = min(distances, key=lambda x: x[1])[0]
    id_to_selected_genre[track_id] = selected_genre

In [33]:
# Replace genre of tracks with multiple genres by the genre with the smallest distance
for track_id, genre in id_to_selected_genre.items():
    unique_trackids.at[track_id, 'track_genre'] = genre

In [ ]:
unique_trackids[unique_trackids['track_genre'] == 'disney']

In [34]:
# Save to csv
unique_trackids.reset_index(inplace=True)
unique_trackids.to_csv('data/spotify_clean.csv', header=True)